In [10]:
#Импортируем необходимые библиотеки

import numpy as np
import pandas as pd

import yargy
from yargy.pipelines import caseless_pipeline
from yargy import Parser

import natasha

from natasha import (
    Segmenter,
    MorphVocab,  
    PER,
    NamesExtractor,
    NewsNERTagger,   
    NewsEmbedding,
    Doc
    )

data = pd.read_csv('test_data.csv')

In [11]:
data.head() #обзор первых строк датасета

,dlg_id,line_n,role,text
0,0,0,client,Алло
1,0,1,manager,Алло здравствуйте
2,0,2,client,Добрый день
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...
4,0,4,client,Ага


In [12]:
def describe_dialogue(num):
    temp_df = data[data.dlg_id == num]
    
    #Флаги
    no_hi, no_intro, no_company, no_bye = True, True, True, True
    
    #ищем, как менеджер поздоровался с клиентом
    GREETINGS = caseless_pipeline(['добрый день', 'здравствуйте', 'добрый вечер']) #включаем несколько вариантов
    parser = Parser(GREETINGS)    
    text = temp_df[(temp_df.role == 'manager')].head().text.values #ищем в первых нескольких репликах менеджера
    
    for line in text:
        for match in parser.findall(line.lower()):
            no_hi = False
            print(f'Приветствие менеджера: {line}', '_'*20, sep='\n') 
            break #как только находим, прерываем цикл
            
    #ищем, как менеджер представился. Используем natasha для поиска именованных сущностей 
    
    emb = NewsEmbedding()
    segmenter = Segmenter()
    morph_vocab = MorphVocab()
    ner_tagger = NewsNERTagger(emb)
    names_extractor = NamesExtractor(morph_vocab)
    
    for line in temp_df[temp_df.role == 'manager'].head().text.values:
        if ' зовут ' in line or 'это ' in line:                  
            doc = Doc(line.title())
            doc.segment(segmenter)
            doc.tag_ner(ner_tagger)
 
            for span in doc.spans:
                span.normalize(morph_vocab)
            {_.text: _.normal for _ in doc.spans}
 
            for span in doc.spans:
                if span.type == PER:
                    span.extract_fact(names_extractor)
   
            if {_.normal for _ in doc.spans if _.fact}:
                no_intro = False
                print('Строка с именем менеджера: ', *{_.normal for _ in doc.spans if _.fact})
                print('_'*20)
        
    #извлекаем имя компании
    COMPANY_NAME = caseless_pipeline(['компания', 'компании'])
    parser = Parser(COMPANY_NAME)
    
    text = temp_df[(temp_df.role == 'manager')].head().text.values
    
    for line in text:
        for match in parser.findall(line.lower()):
            no_company = False
            print(f'Строка с названием компании: {line}', '_'*20, sep='\n')
            break
    
    #ищем, как менеджер попрощался с клиентом    
    GOODBYES = caseless_pipeline(['до свидания', 'всего хорошего', 'всего доброго', 'хорошего дня', 'хорошего вечера'])
    parser = Parser(GOODBYES)
    
    text = temp_df[(temp_df.role == 'manager')].tail(10).text.values #ищем в последних нескольких репликах
    
    for line in text:
        for match in parser.findall(line.lower()):
            no_bye = False
            print(f'Строка, где менеджер попрощался с клиентом: {line}', '_'*20, sep='\n')
            break
    
    if no_hi:
        print('Менеджер не поздоровался', '_'*20, sep='\n')
    if no_intro:
        print('Менеджер не представился', '_'*20, sep='\n')
    if no_company:
        print('Менеджер не назвал компанию', '_'*20, sep='\n')
    if no_bye:
        print('Менеджер не попрощался', '_'*20, sep='\n')

In [13]:
describe_dialogue(0) #тестируем скрипт

Приветствие менеджера: Алло здравствуйте
____________________
Строка с именем менеджера:  Зовут Ангелина
____________________
Строка с названием компании: Меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчивается
____________________
Строка, где менеджер попрощался с клиентом: Всего хорошего до свидания
____________________
